# Neural Networks

In [1]:
import pandas as pd
from joblib import dump
from pathlib import Path
import numpy as np

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score

data_path = Path.home()/'OneDrive'/'Kirstin'/'Uni'/'Year4'/'MSciProject'/'data_files'/'saved_files'

### Initialising

In [2]:
# initialising WandB
import wandb, os

os.environ["WANDB_API_KEY"] = "e84d2e19bd2cc42ec6e5d232cd0b6f0fe41f2189"
os.environ["WANDB_NOTEBOOK_NAME"] = "NN_models.ipynb"


# Syntax for using WandB:

# wandb.init(project="MSciProject", name="name", notebook="your-notebook-name")
# code here
# wandb.finish()

In [2]:
# Load data
data = pd.read_csv(data_path/'for_model.csv', parse_dates=['time'])

data.sample(5)

,time,flag,u10_0,u10_1,u10_2,u10_3,u10_4,u10_5,u10_6,u10_7,...,v500_2_past,v500_3_past,v500_4_past,v500_5_past,v500_6_past,v500_7_past,v500_8_past,v500_13_past,v500_14_past,v500_15_past
28098,2008-12-09 02:08:00,1.0,5.146151,10.848257,5.385287,6.261186,0.190907,0.380253,-0.938153,6.941427,...,-7.298292,-4.455450,1.694595,-33.181866,-35.525610,-35.092330,-31.529179,-29.741205,-13.049791,-19.230917
63850,2019-02-23 18:36:00,0.0,-0.080545,10.669509,4.786285,1.651852,-0.309001,-4.104822,-5.957775,-6.157590,...,37.456635,31.843380,21.519958,13.360619,13.154780,13.204849,32.052925,-0.721662,27.887953,16.359177
20806,2006-12-12 11:06:00,0.0,13.843216,12.023794,3.466795,11.710372,9.169489,9.846270,10.730026,15.995771,...,4.419672,6.579529,-3.107877,0.438633,0.024874,2.641624,6.819158,-6.908395,-6.748642,5.188082
66730,2020-02-02 05:41:00,0.0,10.819938,-4.883079,-0.568672,-4.720872,11.069313,9.390392,10.015724,11.489231,...,-9.225391,8.064178,2.862093,9.310478,20.206694,17.979542,0.770900,7.005633,23.918076,16.001648
19511,2006-07-18 22:27:00,0.0,-3.396057,-4.546572,-2.404162,-1.505665,-2.673920,-3.262483,-1.488968,-2.701052,...,8.685511,5.925582,7.741770,12.810699,10.984708,18.689693,15.693607,11.168287,-0.210959,22.826468


In [3]:
# Convert "time" column to datetime format
#data['time'] = pd.to_datetime(data['time'], format='%d/%m/%Y %H:%M')

# Split the data into training and testing sets based on the date
train_data = data[(data['time'].dt.year >= 2017) & (data['time'].dt.year <= 2019)]
test_data = data[(data['time'].dt.year >= 2020) & (data['time'].dt.year <= 2022)]

print(f"Train range: {train_data['time'].min()} -> {train_data['time'].max()}. Length: {len(train_data)}")
print(f"Test range: {test_data['time'].min()} -> {test_data['time'].max()}. Length: {len(test_data)}")

# saving the date ranges for WandB tracking
training_date_range = "2017-01-01 to 2019-12-31"
testing_date_range = "2020-01-01 to 2022-12-31"

# Drop the "time" column as it won't be used in the model
train_data = train_data.drop(columns=['time'])
test_data = test_data.drop(columns=['time'])

# Define the features (X) and the target (y)
X_train = train_data.drop(columns=['flag'])
y_train = train_data['flag']
X_test = test_data.drop(columns=['flag'])
y_test = test_data['flag']

# Balanced Data - removing NaN values and associated data
y_train = y_train.dropna()
y_test = y_test.dropna()

X_train = X_train.loc[y_train.index]
X_test = X_test.loc[y_test.index]

Train range: 2017-01-01 00:01:00 -> 2019-12-31 22:31:00. Length: 9942
Test range: 2020-01-01 00:41:00 -> 2022-12-31 21:57:00. Length: 9561


### Models

#### Default Parameters

In [22]:
# setting up a neural network model with default parameters
nn_model = MLPClassifier(max_iter=1000, random_state=42)

nn_model.fit(X_train, y_train)

# Predictions
y_pred_nn_test = nn_model.predict(X_test)
y_pred_nn_train = nn_model.predict(X_train)

# calculating scores
precision_test = precision_score(y_test, y_pred_nn_test)
precision_train = precision_score(y_train, y_pred_nn_train)
recall_test = recall_score(y_test, y_pred_nn_test)
recall_train = recall_score(y_train, y_pred_nn_train)
f1_test = f1_score(y_test, y_pred_nn_test)
f1_train = f1_score(y_train, y_pred_nn_train)

print(f"Precision on Training Set = {precision_train:.3f}")
print(f"Precision on Testing Set = {precision_test:.3f}")
print(f"Recall on Training Set = {recall_train:.3f}")
print(f"Recall on Testing Set = {recall_test:.3f}")
print(f"F1 Score on Training Set = {f1_train:.3f}")
print(f"F1 Score on Testing Set = {f1_test:.3f}")


Precision on Training Set = 0.262
Precision on Testing Set = 0.319
Recall on Training Set = 0.988
Recall on Testing Set = 0.989
F1 Score on Training Set = 0.414
F1 Score on Testing Set = 0.483


#### Grid Search for Hyperparameter Tuning

In [23]:
nn_classifier = MLPClassifier()
params = nn_classifier.get_params()

# printing default parameters
print(params)

{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 200, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}


In [13]:
# hyperparameter tuning

# Define the model
model = MLPClassifier(max_iter=1000, hidden_layer_sizes=(50, 50, 50), 
                         activation='relu', solver='adam', alpha=0.0001, random_state=42,
                         learning_rate='constant', batch_size=100, early_stopping=False)

# Define the hyperparameter grid
'''
param_grid = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
    'batch_size': [100, 200, 300],
    'max_iter': [1000, 2000],
    'early_stopping': [True, False]
}
'''
random_states = np.linspace(0,100,101)
random_states = random_states.astype(int)

param_grid = {
    'random_state': random_states
}

# Define the grid search
grid_search = GridSearchCV(model, param_grid, n_jobs=-1, scoring='f1', cv=5)

# Fit the grid search
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
best_f1 = grid_search.best_score_
print(f"Best parameters: {best_params}")
print(f"Best F1 score: {best_f1:.3f}")

Best parameters: {'random_state': 11}
Best F1 score: 0.560


#### Exploring Optimised Hyperparameters

In [17]:
# wandb.init(project="NeuralNetworks")

nn_model = MLPClassifier(max_iter=1000, hidden_layer_sizes=(50, 50, 50), 
                         activation='relu', solver='adam', alpha=0.0001, random_state=11,
                         learning_rate='constant', batch_size=100, early_stopping=False, shuffle=False)

nn_model.fit(X_train, y_train)

# Predictions
y_pred_nn_test = nn_model.predict(X_test)
y_pred_nn_train = nn_model.predict(X_train)

# calculating scores
precision_test = precision_score(y_test, y_pred_nn_test)
precision_train = precision_score(y_train, y_pred_nn_train)
recall_test = recall_score(y_test, y_pred_nn_test)
recall_train = recall_score(y_train, y_pred_nn_train)
f1_test = f1_score(y_test, y_pred_nn_test)
f1_train = f1_score(y_train, y_pred_nn_train)

print(f"Precision on Training Set = {precision_train:.3f}")
print(f"Precision on Testing Set = {precision_test:.3f}")
print(f"Recall on Training Set = {recall_train:.3f}")
print(f"Recall on Testing Set = {recall_test:.3f}")
print(f"F1 Score on Training Set = {f1_train:.3f}")
print(f"F1 Score on Testing Set = {f1_test:.3f}")

wandb.log({"model_name":"Neural Network", "training_precision":precision_train, "testing_precision":precision_test, 
            "training_recall":recall_train, "testing_recall":recall_test, "training_f1":f1_train, "testing_f1":f1_test,
            "training date range": training_date_range, "testing date range": testing_date_range})

wandb.finish()

Precision on Training Set = 0.469
Precision on Testing Set = 0.556
Recall on Training Set = 0.744
Recall on Testing Set = 0.737
F1 Score on Training Set = 0.575
F1 Score on Testing Set = 0.634
